In [1]:
import pandas as pd
import os
import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import style
# import seaborn as sns
# import pylab
import random 
from pandas import DataFrame
import re
from glob import glob
# import matplotlib.pyplot as plt
import math
from statistics import mode
from collections import Counter
from decimal import getcontext
from datetime import date
from datetime import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import BDay

In [2]:
filepath = "C:/Users/janew/Desktop/DA Project/Quantium Project/Task 2/"
qviData = pd.read_csv(filepath+"QVI_data.csv")

In [3]:
'''
Trial stores: 76, 86, 88
match the following metrics:
  - monthly overall sales revenue
  - monthly number of customers
  - monthly number of transactions per customer
'''

'\nTrial stores: 76, 86, 88\nmatch the following metrics:\n  - monthly overall sales revenue\n  - monthly number of customers\n  - monthly number of transactions per customer\n'

In [4]:
# create a column to store the date in int format
qviData['DATE'] = qviData['DATE'].astype(str)
qviData['year'] = qviData['DATE'].astype(str).str[:4]
qviData['month'] = qviData['DATE'].astype(str).str[5:7]
qviData['date'] = qviData['year']+qviData['month']
qviData = qviData.drop(columns = ['year','month'])
qviData['date'] = pd.to_numeric(qviData['date'])

In [5]:
# monthly total sales
# monthly number of customers
totalSales = qviData.groupby(['STORE_NBR', 'date'])['TOT_SALES'].sum().reset_index()

nCustomers = qviData.groupby(['STORE_NBR', 'date']).agg({'LYLTY_CARD_NBR':['nunique']}).reset_index()
nCustomers.reset_index(inplace=False)
nCustomers = nCustomers.droplevel(level = 1, axis =1)

In [6]:
# momthly number of transactions per customer
nTxnPerCust= qviData.groupby(['STORE_NBR', 'date']).agg({'TXN_ID':['nunique'],'LYLTY_CARD_NBR':['nunique']}, index = False)
nTxnPerCust.reset_index(inplace=True)
nTxnPerCust = nTxnPerCust.droplevel(level = 1, axis =1)
nTxnPerCust['AvgTxnCustomer'] = round(nTxnPerCust['TXN_ID']/nTxnPerCust['LYLTY_CARD_NBR'],4)

In [7]:
# monthly number of chips per transaction
nChipsPerTxn = qviData.groupby(['STORE_NBR', 'date']).agg({'PROD_QTY':['sum'], 'TXN_ID':['nunique']}, index = False)
nChipsPerTxn.reset_index(inplace = True)
nChipsPerTxn = nChipsPerTxn.droplevel(level = 1, axis =1)
nChipsPerTxn['nChipsPerTxn'] = round(nChipsPerTxn['PROD_QTY']/nChipsPerTxn['TXN_ID'], 4)

In [8]:
# merge all raw data
temp = totalSales.merge(nCustomers, on = ['STORE_NBR','date'])
temp1 = temp.merge(nTxnPerCust, on = ['STORE_NBR','date','LYLTY_CARD_NBR'])
temp2 = temp1.merge(nChipsPerTxn, on = ['STORE_NBR','date','TXN_ID'])
qvi_all = temp2
qvi_all = qvi_all[['STORE_NBR', 'date', 'TOT_SALES', 'PROD_QTY', 'TXN_ID', 'LYLTY_CARD_NBR',
                  'AvgTxnCustomer', 'nChipsPerTxn']]

In [10]:
# check if all stores have all 12 months full obvervation periods
qvi_period = qvi_all.groupby(['STORE_NBR']).agg({'date':['nunique']})
qvi_period.reset_index(inplace = True)
qvi_period = qvi_period.droplevel(level = 1, axis =1)

In [11]:
qvi_period = qvi_period[qvi_period['date']!=12]

In [12]:
ls = qvi_period['STORE_NBR'].to_list()
ls

[11, 31, 44, 76, 85, 92, 117, 193, 206, 211, 218, 252]

In [13]:
qvi_all_new = qvi_all[~qvi_all['STORE_NBR'].isin(ls)]

,STORE_NBR,date,TOT_SALES,PROD_QTY,TXN_ID,LYLTY_CARD_NBR,AvgTxnCustomer,nChipsPerTxn
0,1,201807,206.9,62,52,49,1.0612,1.1923
1,1,201808,176.1,54,43,42,1.0238,1.2558
2,1,201809,278.8,75,62,59,1.0508,1.2097
3,1,201810,188.1,58,45,44,1.0227,1.2889
4,1,201811,192.6,57,47,46,1.0217,1.2128
...,...,...,...,...,...,...,...,...
3164,272,201902,395.5,91,48,45,1.0667,1.8958
3165,272,201903,442.3,101,53,50,1.0600,1.9057
3166,272,201904,445.1,105,55,54,1.0185,1.9091
3167,272,201905,314.6,71,40,34,1.1765,1.7750


In [14]:
# select trial period
qvi_all_new = qvi_all_new[qvi_all_new['date'] < 201902]
qvi_all_new = qvi_all_new.reset_index(drop=True)
qvi_all_new

,STORE_NBR,date,TOT_SALES,PROD_QTY,TXN_ID,LYLTY_CARD_NBR,AvgTxnCustomer,nChipsPerTxn
0,1,201807,206.9,62,52,49,1.0612,1.1923
1,1,201808,176.1,54,43,42,1.0238,1.2558
2,1,201809,278.8,75,62,59,1.0508,1.2097
3,1,201810,188.1,58,45,44,1.0227,1.2889
4,1,201811,192.6,57,47,46,1.0217,1.2128
...,...,...,...,...,...,...,...,...
1815,272,201809,304.7,71,36,32,1.1250,1.9722
1816,272,201810,430.6,99,50,44,1.1364,1.9800
1817,272,201811,376.2,87,45,41,1.0976,1.9333
1818,272,201812,403.9,89,47,47,1.0000,1.8936


In [ ]:
'''
Select control store to compare to trial stores:
    method: calculate how correlated the performance of each store is to the trial store
'''

In [15]:
# write a function so that no need to calculate the corelation for each trial store
# loop through each control store
qvi_all_new.corr(method ='pearson')

,STORE_NBR,date,TOT_SALES,PROD_QTY,TXN_ID,LYLTY_CARD_NBR,AvgTxnCustomer,nChipsPerTxn
STORE_NBR,1.000000e+00,3.219924e-17,-0.039549,-0.039115,-0.044674,-0.049747,-0.026642,-0.009236
date,3.219924e-17,1.000000e+00,0.000280,0.000835,-0.003826,-0.004974,-0.002468,0.009021
TOT_SALES,-3.954877e-02,2.799127e-04,1.000000,0.974984,0.970007,0.974824,0.786726,0.686128
PROD_QTY,-3.911518e-02,8.348341e-04,0.974984,1.000000,0.993582,0.980884,0.873941,0.689521
TXN_ID,-4.467428e-02,-3.826254e-03,0.970007,0.993582,1.000000,0.991500,0.866756,0.625563
LYLTY_CARD_NBR,-4.974651e-02,-4.974466e-03,0.974824,0.980884,0.991500,1.000000,0.802060,0.619553
AvgTxnCustomer,-2.664164e-02,-2.468108e-03,0.786726,0.873941,0.866756,0.802060,1.000000,0.574355
nChipsPerTxn,-9.235719e-03,9.021404e-03,0.686128,0.689521,0.625563,0.619553,0.574355,1.000000


In [ ]:
'''
Conclusion
We’ve found control stores 233, 155, 237 for trial stores 77, 86 and 88 respectively.
The results for trial stores 77 and 88 during the trial period show a significant difference in 
at least two of the three trial months but this is not the case for trial store 86. We can check 
with the client if the implementation of the trial was different in trial store 86 but overall, 
the trial shows a significant increase in sales. 
Now that we have finished our analysis, we can prepare our presentation to the Category Manager.
'''